In [20]:
import nltk
import pandas as pd
from nltk.util import ngrams
from nltk.metrics import jaccard_distance
from collections import Counter
from sentence_transformers import SentenceTransformer, CrossEncoder, util

In [7]:
data = pd.read_excel("output.xlsx")

In [12]:
data.head()

,brand,child_category,parent category,offer,retailer
0,prego,red pasta sauce,pasta sauce,,
1,ragu,red pasta sauce,pasta sauce,,
2,classico,red pasta sauce,pasta sauce,,
3,heb,red pasta sauce,pasta sauce,,
4,barilla,red pasta sauce,pasta sauce,barilla® pesto sauce,


### Create String for Encoding

Sentence Transformer is sensitive to sentence instead of single word, so I aggregated variables into sentences within each row, improving overall performance

In [13]:
offer_embedding_string = []
for index, row in data.iterrows():
    sentence =  'for the brand: ' + row['brand'] + ', the retailer is ' + row['retailer'] + ', the product category is ' + row['child_category'] + ', which belongs to parent category ' + row['parent category'] + ', now the offer is: ' + row['offer']
    offer_embedding_string.append(sentence)

In [75]:
offer_embedding_string[0:5]

['for the brand: prego, the retailer is , the product category is red pasta sauce, which belongs to parent category pasta sauce, now the offer is: ',
 'for the brand: ragu, the retailer is , the product category is red pasta sauce, which belongs to parent category pasta sauce, now the offer is: ',
 'for the brand: classico, the retailer is , the product category is red pasta sauce, which belongs to parent category pasta sauce, now the offer is: ',
 'for the brand: heb, the retailer is , the product category is red pasta sauce, which belongs to parent category pasta sauce, now the offer is: ',
 'for the brand: barilla, the retailer is , the product category is red pasta sauce, which belongs to parent category pasta sauce, now the offer is: barilla® pesto sauce']

### Encoding via Sentence Transformer

In [21]:
bi_encoder = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')
bi_encoder.max_seq_length = 256
top_k = 32

In [23]:
offer_embeddings = bi_encoder.encode(offer_embedding_string, convert_to_tensor=True, show_progress_bar=True)

Batches:   0%|          | 0/325 [00:00<?, ?it/s]

In [24]:
offer_embeddings.shape

torch.Size([10376, 384])

### Create String for Viewing

To enhance user readability, I designed a more straightforward, user-friendly output string

In [17]:
offer_user_string = []
for index, row in data.iterrows():
    sentence =  'brand: ' + row['brand'] + ' | retailer: ' + row['retailer'] + ' | category: ' + row['child_category'] + ' | parent category: ' + row['parent category'] + ' | offer: ' + row['offer']
    offer_user_string.append(sentence)

In [76]:
offer_user_string[0:5]

['brand: prego | retailer:  | category: red pasta sauce | parent category: pasta sauce | offer: ',
 'brand: ragu | retailer:  | category: red pasta sauce | parent category: pasta sauce | offer: ',
 'brand: classico | retailer:  | category: red pasta sauce | parent category: pasta sauce | offer: ',
 'brand: heb | retailer:  | category: red pasta sauce | parent category: pasta sauce | offer: ',
 'brand: barilla | retailer:  | category: red pasta sauce | parent category: pasta sauce | offer: barilla® pesto sauce']

Mapped `offer_embedding_string` to `offer_user_string`, so that the algorithm could convert embedding strings into user-friendly output

In [25]:
mapping = {}
for i in range(len(offer_embedding_string)):
    key = offer_embedding_string[i]
    value = offer_user_string[i]
    mapping[key] = value

### Baseline Algorithm: Cross Encoder Scoring Mechanism

1. Function refer to sentence transformer repo:
https://github.com/UKPLab/sentence-transformers/blob/master/examples/applications/retrieve_rerank/retrieve_rerank_simple_wikipedia.ipynb

2. The main idea is to use bi-encoder to encode data, and use cross-encoder to get score and rank the results

In [66]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

In [53]:
def baseline_search(query):

    question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
    hits = util.semantic_search(question_embedding, offer_embeddings, top_k=top_k)
    hits = hits[0]  # Get the hits for the first query
    cross_inp = [[query, offer_embedding_string[hit['corpus_id']]] for hit in hits]
    cross_scores = cross_encoder.predict(cross_inp)

    # Sort results by the cross-encoder scores
    for idx in range(len(cross_scores)):
        hits[idx]['cross-score'] = cross_scores[idx]

    hits = sorted(hits, key=lambda x: x['cross-score'], reverse=True)

    res = []
    for hit in hits[0:3]:
        embed_str = offer_embedding_string[hit["corpus_id"]]
        user_str = mapping[embed_str]
        res.append((hit['cross-score'], user_str))


    return res

#### challenges

The current scoring mechanism presents two major challenges:

1. **"No-offer Problem"**: Results without offers are prioritized in the top rank, while results with offers may not appear prominently. This can lead to users missing important offer information.

2. **"Typo Problem"**: The algorithm lacks flexibility in handling typos. For example, if a user types "Targt," the algorithm cannot correctly identify the retailer "target" and provide relevant results.

#### examples

Example illustrating "No-offer Problem": 
Both the first and second results do not contain any offers

In [54]:
baseline_search('Target')

[(4.451646,
  'brand: target | retailer:  | category: deodorant & antiperspirant | parent category: health & wellness | offer: '),
 (4.116417,
  'brand: target | retailer:  | category: wine | parent category: alcohol | offer: '),
 (3.2561677,
  'brand: back to the roots | retailer: target | category: packaged meals & sides | parent category: pantry | offer: back to the roots raised bed gardening kit with soil, seeds and plant food, at target')]

Example illustrating "Typo Problem": Suppose the user intends to search for 'target' offers but types 'targt,' the results fails to display the 'target' offers

In [55]:
baseline_search('Targt')

[(-5.43099,
  'brand: tarot | retailer:  | category: wine | parent category: alcohol | offer: '),
 (-5.8334913,
  'brand: tarima | retailer:  | category: wine | parent category: alcohol | offer: '),
 (-6.839417,
  'brand: earth rider blackbecrush tart wheat ale | retailer:  | category: beer | parent category: alcohol | offer: ')]

### Improved Algorithm: Ngram Fuzzy Search + Cross Encoder + Rerank Scoring Mechanism

To address the two issues mentioned earlier, I have made improvements as follows:

1. **"Typo Problem"**: Introduced Ngram fuzzy search to improve typo identification, ensuring better accuracy in recognizing user queries with typo errors


2. **"No-offer Problem"**: Add a rank function and designed specific reranking strategy:
    - Select the top 5 search results based on relevance
    - Filter all results to identify the top 5 results that include offers
    - Combine the top 5 search results and top 5 filtered results,  rerank them based on their scores to generate the final output

In [44]:
def deduplicate(input_list):
    return sorted(set(input_list))

In [45]:
def get_obj_string_list(name):
    obj_string = []
    for index, row in data.iterrows():
        sentence =  row[name] 
        if not pd.isna(sentence) and sentence != "":
            obj_string.append(sentence.lower())
    return deduplicate(obj_string)

In [46]:
def get_ngrams(name, n):
    characters = [c for c in name.lower()]
    return list(ngrams(characters, n))

In [67]:
def ngram_fuzzy_search(search_text):
    retail_string = get_obj_string_list("retailer")
    brand_string = get_obj_string_list("brand")
    cat_string = deduplicate(get_obj_string_list("child_category") + get_obj_string_list("parent category"))
    entry_string = deduplicate(cat_string + brand_string + retail_string)
    
    input_text = search_text
    n = 2
    input_ngrams = get_ngrams(input_text, n)
    similarities = []
    
    for entry in entry_string:
        entry_ngrams = get_ngrams(entry, n)
        similarity = 1 - jaccard_distance(set(input_ngrams), set(entry_ngrams))
        similarities.append((similarity, entry))
        
    threshold = 0.3
    k = 3
    filtered_similarities = [(similarity, entry) for similarity, entry in similarities if similarity >= threshold]
    sorted_similarities = sorted(filtered_similarities, key=lambda x: x[0], reverse=True)
    queries = sorted_similarities[:k]
#     input_text = "targt"
    return queries

In [68]:
def cross_encode(encode_name):
    queries = ngram_fuzzy_search(encode_name)
    res = []

    for query_sim, query in queries:
        question_embedding = bi_encoder.encode(query, convert_to_tensor=True)
        hits = util.semantic_search(question_embedding, offer_embeddings, top_k=top_k)
        hits = hits[0]  # Get the hits for the first query
        cross_inp = [[query, offer_embedding_string[hit['corpus_id']]] for hit in hits]
        cross_scores = cross_encoder.predict(cross_inp)

        # Sort results by the cross-encoder scores
        for idx in range(len(cross_scores)):
            hits[idx]['cross-score'] = cross_scores[idx]
        
        for hit in hits:
            embed_str = offer_embedding_string[hit["corpus_id"]]
            user_str = mapping[embed_str]
            have_offer =  not user_str.strip().endswith("offer:")
            res.append((hit['cross-score'], user_str, have_offer, query_sim, query))
            
    res = sorted(res, key=lambda x:x[0], reverse=True)
    return res

In [69]:
def my_search(search_name):
    DEBUG=False
    offer_list = cross_encode(search_name)
    res = []
    updated_offer_list = []
    for offer in offer_list:
        cross_score, user_str, have_offer, query_score, query = offer
        combined_score = cross_score + 10*query_score
        updated_offer_list.append((combined_score, have_offer, user_str, query))        
    
    updated_offer_list = sorted(updated_offer_list, key=lambda x: x[0], reverse=True)
    if DEBUG:
        print("===============VIS================")
        for offer in updated_offer_list[:20]:
            print("score:", offer[0], "have_offer:", offer[1], "user_str:", offer[2])
            print("=====")
    
    with_offer = [offer for offer in updated_offer_list if offer[1]]
    with_offer_sorted = sorted(with_offer,key=lambda x: x[0], reverse=True)
    for offer in with_offer_sorted[0:5]:
        res.append((offer[0],offer[2]))
    
    for offer in updated_offer_list[0:5]:
        if (offer[0],offer[2]) in res:
            continue
        res.append((offer[0], offer[2]))
                
    return list(res)

#### examples

Example addressing "No-offer Problem": records with offers now appear prominently at the top of the search results

In [70]:
my_search('target')

[(13.256167650222778,
  'brand: back to the roots | retailer: target | category: packaged meals & sides | parent category: pantry | offer: back to the roots raised bed gardening kit with soil, seeds and plant food, at target'),
 (12.702252864837646,
  'brand: back to the roots | retailer: target | category: packaged meals & sides | parent category: pantry | offer: back to the roots organic kits and planters, at target'),
 (12.657567977905273,
  "brand: loreal paris cosmetics | retailer: target | category: makeup | parent category: beauty | offer: l'oreal paris true match foundation at target"),
 (12.657567977905273,
  "brand: loreal paris cosmetics | retailer: target | category: makeup | parent category: beauty | offer: l'oréal paris true match foundation at target"),
 (12.459017992019653,
  'brand: dove | retailer: target | category: hair care | parent category: health & wellness | offer: dove hand wash, select varieties at target'),
 (14.451645851135254,
  'brand: target | retailer: 

Example addressing "Typo Problem": the algorithm can now identify typographical errors made by users and return the relevant results

In [71]:
my_search('targt')

[(8.256167650222778,
  'brand: back to the roots | retailer: target | category: packaged meals & sides | parent category: pantry | offer: back to the roots raised bed gardening kit with soil, seeds and plant food, at target'),
 (7.7022528648376465,
  'brand: back to the roots | retailer: target | category: packaged meals & sides | parent category: pantry | offer: back to the roots organic kits and planters, at target'),
 (7.657567977905273,
  "brand: loreal paris cosmetics | retailer: target | category: makeup | parent category: beauty | offer: l'oreal paris true match foundation at target"),
 (7.657567977905273,
  "brand: loreal paris cosmetics | retailer: target | category: makeup | parent category: beauty | offer: l'oréal paris true match foundation at target"),
 (7.459017992019653,
  'brand: dove | retailer: target | category: hair care | parent category: health & wellness | offer: dove hand wash, select varieties at target'),
 (9.451645851135254,
  'brand: target | retailer:  | ca

Example for typing category

In [72]:
my_search('pantry')

[(10.72649073600769,
  'brand: annies homegrown grocery | retailer:  | category: soup & broth | parent category: pantry | offer: any general mills™ products, buy 2\ngood rewards members only'),
 (10.667452931404114,
  'brand: annies homegrown grocery | retailer:  | category: soup & broth | parent category: pantry | offer: general mills™ products, select brands, spend $7 at convenience stores'),
 (10.345320641994476,
  'brand: annies homegrown grocery | retailer:  | category: soup & broth | parent category: pantry | offer: general mills™ products, select varieties, spend $12'),
 (10.079375550150871,
  'brand: annies homegrown grocery | retailer:  | category: cooking & baking | parent category: pantry | offer: general mills™ products, select varieties, spend $12'),
 (-7.405521392822266,
  'brand: gatorade | retailer: dillons grocery | category: sports drinks | parent category: sports drinks & enhanced waters | offer: gatorade® fast twitch®, 12-ounce single serve, at kroger'),
 (11.197189

Example for typing brand

In [73]:
my_search('subway')

[(15.790562629699707,
  'brand: subway | retailer: subway | category: cooking & baking | parent category: pantry | offer: spend $25 at subway'),
 (15.787304878234863,
  'brand: subway | retailer: subway | category: cooking & baking | parent category: pantry | offer: spend $35 at subway'),
 (15.780460357666016,
  'brand: subway | retailer: subway | category: cooking & baking | parent category: pantry | offer: spend $15 at subway'),
 (15.772863388061523,
  'brand: subway | retailer: subway | category: cooking & baking | parent category: pantry | offer: spend $10 at subway'),
 (-0.8557090759277344,
  'brand: caseys general store | retailer: caseys general store | category: frozen pizza & pizza snacks | parent category: frozen | offer: spend $5 on brisket mac & cheese sandwich or other single-serve prepared food item'),
 (-0.28975772857666016,
  'brand: 7th avenue pizza | retailer:  | category: frozen pizza & pizza snacks | parent category: frozen | offer: ')]